In [56]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from itertools import chain
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#torch.cuda.current_device()

In [2]:
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [3]:
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [4]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [5]:
MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)


def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [6]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('eng', 'fra', True)
print(random.choice(pairs))

Reading lines...
Read 135842 sentence pairs
Trimmed to 10599 sentence pairs
Counting words...
Counted words:
fra 4345
eng 2803
['elle est mannequin .', 'she s a model .']


In [9]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, device):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.device = device
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    # input shape: (B, 1)
    # hidden shape: (B, hidden_size)
    # output shape: (B, hidden_size)
    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=self.device)

In [10]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, device):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.device=device
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        #self.softmax = nn.LogSoftmax(dim=1)

    # input shape: (B, hidden size)
    # output shape: (B, hidden size)
    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.out(output[0])
        #output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=self.device)

In [11]:
# number of input and output vocabulary
input_size = input_lang.n_words
output_size = output_lang.n_words
hidden_size = 10
# encoder decoder


In [12]:
# dataset for translation
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
class dataset(Dataset):
    def __init__(self, device):
        super(dataset, self).__init__()
        self.data = pairs
        self.device = device
    
    def __len__(self):
        return len(pairs)
    
    def __getitem__(self, idx):
        input, output = pairs[idx]
        input_token = torch.tensor([input_lang.word2index[word] for word in input.split()] +
                                   [EOS_token],
                                   device=self.device)
        output_token = torch.tensor([SOS_token] + 
                                    [output_lang.word2index[word] for word in output.split()] +
                                    [EOS_token],
                                    device=self.device)
        return input_token, output_token
    
data = dataset(device)
#dataloader = DataLoader(data,batch_size=2)

In [123]:
input_sentence = "je suis mannequin"
input_token = torch.tensor([input_lang.word2index[word] for word in input_sentence.split()] +
                                   [EOS_token],
                                   device=device)


In [118]:
class seq2seq():
    def __init__(self, input_size, hidden_size, output_size, device):
        super(seq2seq, self).__init__()
        self.device = device
        self.encoder = EncoderRNN(input_size, hidden_size, self.device).to(self.device)
        self.decoder = DecoderRNN(hidden_size,output_size, self.device).to(self.device)
    def train(self, dataset, epochs=5):
        criterion = nn.CrossEntropyLoss()
        encoder_optimizer = torch.optim.SGD(self.encoder.parameters(), lr=0.01)
        decoder_optimizer = torch.optim.SGD(self.decoder.parameters(), lr=0.01)
        for epoch in range(epochs):
            epoch_loss = 0
            for token_fra, token_eng in dataset:
                # start running encoder
                encoder_optimizer.zero_grad()
                decoder_optimizer.zero_grad()
                
                encoder_hidden = self.encoder.initHidden()
                decoder_hidden = self.decoder.initHidden()
                loss = 0
                for token in token_fra:
                    encoder_output, encoder_hidden = self.encoder.forward(token
                                                                          ,encoder_hidden)

                decoder_hidden = encoder_output
                for index, token in enumerate(token_eng[0:len(token_eng)-1]):
                    decoder_output, decoder_hidden = self.decoder.forward(token
                                                                          ,decoder_hidden)
                    loss += criterion(decoder_output, torch.tensor([token_eng[index+1]]))
                
                epoch_loss += loss
                loss.backward()
                decoder_optimizer.step()
                encoder_optimizer.step()
            print("epoch:", epoch, epoch_loss/len(dataset))
    # data is a encoded sentence like nn.tensor([13,24,56,...])
    def predict(data):
        with torch.no_grad():
            encoder_hidden = self.encoder.initHidden()
            loss = 0
            for token in input_token:
                encoder_output, encoder_hidden = self.encoder.forward(token
                                                                ,encoder_hidden)
            decoder_hidden = encoder_output
            decoder_output = None
            eng_output = "SOS"
            decoder_token = torch.tensor([SOS_token])
    
            while eng_output != "EOS":
                decoder_output,decoder_hidden = self.decoder.forward(decoder_token, decoder_hidden)
                decoder_token = torch.argmax(decoder_output)
                eng_output = output_lang.index2word[int(decoder_token.numpy())]
                print(eng_output)

In [119]:
model = seq2seq(input_size=input_lang.n_words, hidden_size=5,output_size=output_lang.n_words, device=device)

In [ ]:
model.train(data,epochs=30)

In [111]:
input_token

tensor([  6,  11, 746,   1])

In [124]:
def predict(data):
    with torch.no_grad():
        encoder_hidden = model.encoder.initHidden()
        loss = 0
        for token in input_token:
            encoder_output, encoder_hidden = model.encoder.forward(token
                                                                ,encoder_hidden)
        decoder_hidden = encoder_output
        decoder_output = None
        eng_output = "SOS"
        decoder_token = torch.tensor([SOS_token])
    
        while eng_output != "EOS":
            decoder_output,decoder_hidden = model.decoder.forward(decoder_token, decoder_hidden)
            decoder_token = torch.argmax(decoder_output)
            eng_output = output_lang.index2word[int(decoder_token.numpy())]
            print(eng_output)

In [125]:
predict(input_token)

i
m
going
to
the
the
be
you
.
EOS
